In [2]:
import pandas as pd
import numpy as np
import random

# 1. Veriyi pkl dosyasından yükle
file_path = 'hazirlanmis_satin_alma.pkl'
data = pd.read_pickle(file_path)

# İlk birkaç satırı görüntüle
print("Veri Setindeki İlk 5 Satır:")
print(data.head())

print("\n")


Veri Setindeki İlk 5 Satır:
   User_ID  Product_ID  Quantity            Timestamp
0       76       67785         4  2024-02-28 15:46:28
1       62       44238         5  2024-03-03 03:48:28
2       99       38560         3  2024-08-03 20:02:49
3       67       48066         1  2024-08-08 04:40:07
4       93       25530         1  2024-06-12 00:33:24




In [3]:
# Ürün havuzunu oluştur
product_pool = data['Product_ID'].unique()


In [12]:
# Kullanıcıları benzersiz olarak listele
unique_users = data['User_ID'].unique()

# Rastgele öneri fonksiyonu
def random_recommendations(user_id, product_pool, n=5):
    return {
        "user_id": user_id,
        "recommended_items": random.sample(list(product_pool), n)
    }

# Tüm kullanıcılar için öneriler oluştur
recommendation_list = [
    random_recommendations(user_id, product_pool) for user_id in unique_users
]

# Önerileri DataFrame olarak düzenle
recommendation_df = pd.DataFrame(recommendation_list)


In [13]:
# DataFrame'in tamamını veya ilk birkaç satırını yazdırabilirsiniz
print(recommendation_df)  # Tüm DataFrame'i yazdırır


    user_id                    recommended_items
0        76  [40361, 65247, 71849, 43507, 28106]
1        62  [36749, 22626, 23563, 30940, 31756]
2        99  [75528, 32846, 24665, 53689, 42028]
3        67  [42648, 70085, 47134, 41206, 24984]
4        93  [61916, 64319, 40293, 74102, 68902]
..      ...                                  ...
95       80  [54023, 22102, 75676, 32155, 55192]
96       46  [30902, 42424, 43365, 36866, 76466]
97       71  [29899, 30562, 49703, 46798, 23594]
98       97  [39910, 26843, 27948, 49867, 48362]
99       13  [40792, 67785, 67122, 22312, 25364]

[100 rows x 2 columns]


In [15]:
# Gerçek etkileşimleri hazırlama
actual_interactions = {
    user_id: group['Product_ID'].tolist()
    for user_id, group in data.groupby('User_ID')
}

# MAP hesaplama fonksiyonu
def calculate_map(recommendations, actual_interactions, k=10):
    map_score = 0
    user_count = 0

    for user_id, recommended_items in recommendations.items():
        relevant_items = actual_interactions.get(user_id, [])
        if not relevant_items:
            continue

        recommended_items = recommended_items[:k]
        hits = 0
        sum_precision = 0
        for i, item in enumerate(recommended_items):
            if item in relevant_items:
                hits += 1
                precision_at_i = hits / (i + 1)
                sum_precision += precision_at_i

        if hits > 0:
            map_score += sum_precision / hits
        user_count += 1

    return map_score / user_count if user_count > 0 else 0

# Öneri sözlüğünü oluştur
recommendations = recommendation_df.set_index('user_id')['recommended_items'].to_dict()

# MAP değerini hesapla
map_score = calculate_map(recommendations, actual_interactions)
print(f"MAP Değeri: {map_score:.4f}")


MAP Değeri: 0.0285


In [16]:
def calculate_cc(recommendations, product_pool):
    # Önerilen tüm ürünleri bir kümede toplama
    recommended_items = set()
    for items in recommendations.values():
        recommended_items.update(items)
    
    # CC Hesabı
    coverage = len(recommended_items) / len(product_pool)
    return coverage

# CC değerini hesapla
cc_score = calculate_cc(recommendations, product_pool)
print(f"CC Değeri: {cc_score:.4f}")


CC Değeri: 0.4002


In [17]:
import numpy as np

def calculate_mns(recommendations, data):
    # Her ürünün popülerlik skorunu hesapla
    product_popularity = data['Product_ID'].value_counts(normalize=True)
    
    # Ters popülerlik skorunu hesapla (-log(Popularity))
    novelty_scores = -np.log(product_popularity)
    
    # Her kullanıcı için önerilen ürünlerin yenilik skorlarını hesapla
    total_novelty = 0
    total_recommendations = 0
    
    for user_id, recommended_items in recommendations.items():
        user_novelty = novelty_scores.loc[recommended_items].sum()
        total_novelty += user_novelty
        total_recommendations += len(recommended_items)
    
    # Ortalama Yenilik Skoru
    mns_score = total_novelty / total_recommendations if total_recommendations > 0 else 0
    return mns_score

# MNS değerini hesapla
mns_score = calculate_mns(recommendations, data)
print(f"MNS Değeri: {mns_score:.4f}")


MNS Değeri: 6.9008


In [18]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def calculate_mds(recommendations, product_features):
    total_diversity = 0
    user_count = 0
    
    for user_id, recommended_items in recommendations.items():
        # Kullanıcının önerilen ürünlerini seç
        feature_vectors = product_features.loc[recommended_items].values
        
        # Eğer öneriler 1 ürün içeriyorsa çeşitlilik hesaplanamaz
        if len(feature_vectors) < 2:
            continue
        
        # Ürünler arasındaki benzerlik matrisini hesapla
        similarity_matrix = cosine_similarity(feature_vectors)
        
        # Benzerlik matrisinden farklılık skoru çıkar (1 - similarity)
        diversity_matrix = 1 - similarity_matrix
        
        # Ürünler arasındaki ortalama çeşitliliği hesapla
        avg_diversity = np.sum(diversity_matrix) / (len(recommended_items) * (len(recommended_items) - 1))
        total_diversity += avg_diversity
        user_count += 1
    
    # Ortalama Çeşitlilik Skoru
    mds_score = total_diversity / user_count if user_count > 0 else 0
    return mds_score

# Örnek ürün özellik matrisi oluştur
# Her ürünün özelliklerini temsil eden rastgele bir matris
product_features = pd.DataFrame(
    np.random.rand(len(product_pool), 5),  # Örnek olarak 5 özellik
    index=product_pool
)

# MDS değerini hesapla
mds_score = calculate_mds(recommendations, product_features)
print(f"MDS Değeri: {mds_score:.4f}")


MDS Değeri: 0.2361


In [19]:
def calculate_mps(recommendations):
    user_ids = list(recommendations.keys())
    total_similarity = 0
    user_count = len(user_ids)

    # Kullanıcılar arasında önerilerin ortak ürünlerini kontrol et
    for i in range(user_count):
        user_a = user_ids[i]
        recommendations_a = set(recommendations[user_a])
        
        for j in range(i + 1, user_count):
            user_b = user_ids[j]
            recommendations_b = set(recommendations[user_b])
            
            # Ortak ürünlerin oranını hesapla
            intersection = len(recommendations_a & recommendations_b)
            union = len(recommendations_a | recommendations_b)
            similarity = intersection / union if union > 0 else 0
            
            # Farklılık skoru (1 - Benzerlik)
            total_similarity += 1 - similarity

    # Ortalama Kişiselleştirme Skoru
    total_comparisons = user_count * (user_count - 1) / 2
    mps_score = total_similarity / total_comparisons if total_comparisons > 0 else 0
    return mps_score

# MPS değerini hesapla
mps_score = calculate_mps(recommendations)
print(f"MPS Değeri: {mps_score:.4f}")


MPS Değeri: 0.9972


In [20]:
import pickle

# Tüm metrikleri tek bir sözlükte topla
random_based_results = {
    "MAP": map_score,
    "CC": cc_score,
    "MNS": mns_score,
    "MDS": mds_score,
    "MPS": mps_score
}

# Tek bir dosyaya kaydet
with open("random_based_sonuclari.pkl", "wb") as f:
    pickle.dump(random_based_results, f)
